In [65]:
#Load selinimum and automatically install the Chrome Driver
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re

In [66]:
#Setup Chrome driver 
options = Options()
#options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
time.sleep(10)

In [67]:
from bs4 import BeautifulSoup

url = "https://student.msu.edu/psc/public/EMPLOYEE/SA/c/NUI_FRAMEWORK.PT_AGSTARTPAGE_NUI.GBL?CONTEXTIDPARAMS=TEMPLATE_ID%3aPTPPNAVCOL&scname=MSU_AA_SCHEDULE_NEW0&PanelCollapsible=Y"
driver.get(url)
time.sleep(20)

The ids might change. I am using this currently.

In [68]:
Semester = {'Summer 2020': "'SSR_CSTRMPRV_VW_DESCR$8'",
            'Fall 2020': "'SSR_CSTRMPRV_VW_DESCR$span$7'",
            'Spring 21': "'SSR_CSTRMPRV_VW_DESCR$span$6'",
            'Summer 21': "'SSR_CSTRMPRV_VW_DESCR$5'",
            'Fall 21': "'SSR_CSTRMPRV_VW_DESCR$4'",
            'Spring 22': "'SSR_CSTRMPRV_VW_DESCR$span$3'",
            'Summer 22': "'SSR_CSTRMPRV_VW_DESCR$2'",
            'Fall 22': "'SSR_CSTRMPRV_VW_DESCR$1'",
            'Spring 23': "'SSR_CSTRMPRV_VW_DESCR$0'",
            'Summer 23': "'SSR_CSTRMCUR_VW_DESCR$0'", 
            'Fall 23': "'SSR_CSTRMCUR_VW_DESCR$1'",
           'Spring 24': "'SSR_CSTRMCUR_VW_DESCR$2'",
           'Summer 24': "'SSR_CSTRMCUR_VW_DESCR$3'"}

            
url = f"javascript:submitAction_win0(document.win0,{Semester['Spring 23']});"

In [69]:
driver.execute_script(url);
time.sleep(30)

In [77]:
from selenium.webdriver.common.by import By
element = driver.find_element(By.ID, 'MSU_CLSRCH_WRK2_SUBJECT')  
element.send_keys("IAH") #pick cmse for example

In [82]:
url = f"javascript:submitAction_win0(document.win0,'MSU_CLSRCH_WRK_SSR_PB_SEARCH');"
driver.execute_script(url);
time.sleep(20)

Function to get basic classes' info

In [83]:
def get_class_info(soup):
    values = []
    divs = soup.find_all("div", class_="ps-htmlarea")
    for div in divs:
        val = (div.get_text(strip=True))
        values.append(val)
    values = list(filter(lambda x: x != "", values))
    values.pop(0)
    reshaped_list = [values[i:i+6] for i in range(0, len(values), 6)]

    # Create a DataFrame from the reshaped list
    col_names = ['Course', 'Type', 'Section', 'Schedule', 'Dates', 'Instructor']
    df = pd.DataFrame(reshaped_list, columns=col_names)
    df[['Course Code', 'Course Name']] = df['Course'].str.split(':', 1, expand=True)
    df[['Type', 'Units']] = df['Type'].str.split('(', 1, expand=True)
    df[['Section', 'Class Nbr', 'Academic Session']] = df['Section'].str.split('/', 2, expand=True)
    df[['Days', 'Time']] = df['Schedule'].str.split(':', 1, expand=True)
    df[['Units','Status']] = df['Units'].str.split(')',1,expand=True)
    df[['Subject','Course Number']] = df['Course Code'].str.split(' ',1,expand=True)

    df = df.drop(['Course', 'Schedule','Course Code'], axis=1)
    df = df[['Subject','Course Number','Course Name','Type','Units','Status','Section','Class Nbr','Academic Session','Days','Time','Dates','Instructor']]
    df['Units'] = df['Units'].str.extract(r'(\d+(?:\.\d+)?)')
    df['Section'] = df['Section'].str.extract(r'(\d+(?:\.\d+)?)')
    df['Class Nbr'] = df['Class Nbr'].str.extract(r'(\d+(?:\.\d+)?)')
    return df

In [148]:
def get_advanced_info(soup):
    loc = None
    email = None
    values = []
    divs = soup.find_all("div", class_="ps-htmlarea")
    for div in divs:
        val = (div.get_text(strip=True))
        values.append(val)
    values = list(filter(lambda x: x != "", values))
    string = values[5].split('Instructor:')
    loc = string[0]
    a_elements = soup.find_all('a')
    for a_element in a_elements:
        # Check if the 'href' attribute exists
        if 'href' in a_element.attrs:
            # Check if the href attribute contains "mailto:"
            if 'mailto:' in a_element['href']:
                # Extract the email address from the href attribute
                email = a_element['href'].split(':')[1]
                break
    return loc, email

In [149]:
def add_info(df):
    location = []
    emails = []
    for i in range(len(df)):
        element = driver.find_element(By.ID, f"DESCR100$0_row_{i}") 
        element.click()
        time.sleep(2)
        driver.switch_to.frame(0)
        body = driver.page_source
        soup = BeautifulSoup(body, 'html.parser')
        loc, email = get_advanced_info(soup)
        location.append(loc)
        emails.append(email)
        cancel_cmd="javascript:doUpdateParent(document.win0,'#ICCancel');"
        driver.execute_script(cancel_cmd);
        time.sleep(2)
    df['Location'] = location
    df['email'] = emails
    return df

In [150]:
body = driver.page_source
soup = BeautifulSoup(body, 'html.parser')
df = get_class_info(soup)  # getting info on the first page

In [151]:
# get the number of results
result_element = soup.find('span', id='MSU_RSLT_NAV_WK_PTPG_ROWS_GRID')
# Extract the text content
result_text = result_element.get_text(strip=True)
result = int(result_text.split()[0])
pages = (result + 49) // 50# get the number of aggregated pages
pages

5

In [153]:
df = add_info(df)
df

,Subject,Course Number,Course Name,Type,Units,Status,Section,Class Nbr,Academic Session,Days,Time,Dates,Instructor,Location,email
0,IAH,201,United States and the World (D),Recitation,4.00,Wait List,001,16474,Regular Academic Session,Fri,8:00 AM-8:50 AM,01/09/2023 - 04/30/2023,To Be Announced,Holden Hall C135,rehberge@msu.edu
1,IAH,201,United States and the World (D),Recitation,4.00,Wait List,001,16474,Regular Academic Session,Fri,8:00 AM-8:50 AM,01/09/2023 - 04/30/2023,To Be Announced,Holden Hall C135,rehberge@msu.edu
2,IAH,201,United States and the World (D),Lecture,0.00,Wait List,1,32300,Regular Academic Session,Tue Thu,8:30 AM-9:50 AM,01/09/2023 - 04/30/2023,D Rehberger,Holden Hall C135,rehberge@msu.edu
3,IAH,201,United States and the World (D),Lecture,0.00,Wait List,1,32300,Regular Academic Session,Tue Thu,8:30 AM-9:50 AM,01/09/2023 - 04/30/2023,D Rehberger,Wilson Hall C102,rehberge@msu.edu
4,IAH,201,United States and the World (D),Recitation,4.00,Wait List,002,16475,Regular Academic Session,Fri,9:10 AM-10:00 AM,01/09/2023 - 04/30/2023,To Be Announced,Holden Hall C135,rehberge@msu.edu
5,IAH,201,United States and the World (D),Recitation,4.00,Wait List,002,16475,Regular Academic Session,Fri,9:10 AM-10:00 AM,01/09/2023 - 04/30/2023,To Be Announced,Holden Hall C135,rehberge@msu.edu
6,IAH,201,United States and the World (D),Lecture,0.00,Wait List,1,32300,Regular Academic Session,Tue Thu,8:30 AM-9:50 AM,01/09/2023 - 04/30/2023,D Rehberger,Holden Hall C135,rehberge@msu.edu
7,IAH,201,United States and the World (D),Lecture,0.00,Wait List,1,32300,Regular Academic Session,Tue Thu,8:30 AM-9:50 AM,01/09/2023 - 04/30/2023,D Rehberger,Wilson Hall C102,rehberge@msu.edu
8,IAH,201,United States and the World (D),Recitation,4.00,Wait List,003,16476,Regular Academic Session,Fri,10:20 AM-11:10 AM,01/09/2023 - 04/30/2023,To Be Announced,Holden Hall C135,rehberge@msu.edu
9,IAH,201,United States and the World (D),Recitation,4.00,Wait List,003,16476,Regular Academic Session,Fri,10:20 AM-11:10 AM,01/09/2023 - 04/30/2023,To Be Announced,Holden Hall C135,rehberge@msu.edu


In [154]:
for i in range(1,pages):
    button = driver.find_element(By.ID, 'MSU_RSLT_NAV_WK_SEARCH_CONDITION2')
    button.click()
    time.sleep(2)
    body = driver.page_source
    soup = BeautifulSoup(body, 'html.parser')
    df_new = get_class_info(soup)
    df_new = add_info(df_new)
    df = df.append(df_new, ignore_index=True)
df

,Subject,Course Number,Course Name,Type,Units,Status,Section,Class Nbr,Academic Session,Days,Time,Dates,Instructor,Location,email
0,IAH,201,United States and the World (D),Recitation,4.00,Wait List,001,16474,Regular Academic Session,Fri,8:00 AM-8:50 AM,01/09/2023 - 04/30/2023,To Be Announced,Holden Hall C135,rehberge@msu.edu
1,IAH,201,United States and the World (D),Recitation,4.00,Wait List,001,16474,Regular Academic Session,Fri,8:00 AM-8:50 AM,01/09/2023 - 04/30/2023,To Be Announced,Holden Hall C135,rehberge@msu.edu
2,IAH,201,United States and the World (D),Lecture,0.00,Wait List,1,32300,Regular Academic Session,Tue Thu,8:30 AM-9:50 AM,01/09/2023 - 04/30/2023,D Rehberger,Holden Hall C135,rehberge@msu.edu
3,IAH,201,United States and the World (D),Lecture,0.00,Wait List,1,32300,Regular Academic Session,Tue Thu,8:30 AM-9:50 AM,01/09/2023 - 04/30/2023,D Rehberger,Wilson Hall C102,rehberge@msu.edu
4,IAH,201,United States and the World (D),Recitation,4.00,Wait List,002,16475,Regular Academic Session,Fri,9:10 AM-10:00 AM,01/09/2023 - 04/30/2023,To Be Announced,Holden Hall C135,rehberge@msu.edu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,IAH,206,"Self, Society, and Technology (D)",Lecture,0.00,Wait List,2,16545,Regular Academic Session,To Be Announced,None,01/09/2023 - 04/30/2023,P Spinner,Arranged,spinnerp@msu.edu
246,IAH,206,"Self, Society, and Technology (D)",Lecture,0.00,Wait List,2,16545,Regular Academic Session,To Be Announced,None,01/09/2023 - 04/30/2023,P Spinner,Natural Sciences Bldg 145,marchi22@msu.edu
247,IAH,206,"Self, Society, and Technology (D)",Lecture,0.00,Wait List,2,16545,Regular Academic Session,To Be Announced,None,01/09/2023 - 04/30/2023,P Spinner,Arranged,spinnerp@msu.edu
248,IAH,206,"Self, Society, and Technology (D)",Recitation,4.00,Wait List,008,32945,Regular Academic Session,Fri,10:20 AM-11:10 AMFri : 10:20 AM-11:10 AM,01/09/2023 - 04/30/202301/09/2023 - 04/30/2023,A Pfau,Ernst Bessey Hall 314,pfauadam@msu.edu
